<a href="https://colab.research.google.com/github/vatzal58/Test/blob/main/tiffin_calc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
import re
from datetime import datetime, timedelta
from tabulate import tabulate

def filter_whatsapp_chat(filename, sender_name, month, keyword):
    # Open the WhatsApp chat export file
    with open(filename, 'r', encoding='utf-8') as file:
        lines = file.readlines()

    # Define a list to hold filtered messages
    filtered_messages = []

    # Loop through each line in the file
    for line in lines:
        # Regex pattern for date, 12-hour time with AM/PM, sender, and message
        match = re.match(r'^(\d{1,2}/\d{1,2}/\d{2,4}), (\d{1,2}:\d{2}\s[APMapm]{2}) - (.*?): (.*)', line)

        if match:
            # Extract components from the matched pattern
            date_str, time_str, sender, message = match.groups()

            # Parse the date using dd/mm/yyyy format
            try:
                date_obj = datetime.strptime(date_str, "%d/%m/%y")
            except ValueError:
                continue  # Skip lines with unrecognized date formats

            # Parse the time
            time_obj = datetime.strptime(time_str, "%I:%M %p")  # 12-hour format with AM/PM

            # Determine meal category based on time
            # Dinner if between 12pm (noon) and 6pm, otherwise Lunch
            meal_category = "Dinner" if 12 <= time_obj.hour < 18 else "Lunch"

            # Adjust date for Lunch category if time is PM
            adjusted_date = date_obj
            if meal_category == "Lunch" and time_str.lower().endswith('pm'):
                adjusted_date = date_obj + timedelta(days=1)

            # Check if the message meets all filter conditions
            if (sender == sender_name and
                date_obj.strftime("%B") == month and
                re.search(r'\b' + re.escape(keyword) + r'\b', message, re.IGNORECASE)):
                # Add to filtered messages list
                filtered_messages.append([
                    f"{date_str} {time_str}",
                    sender,
                    message,
                    meal_category,
                    adjusted_date.strftime("%d/%m/%y")
                ])

    return filtered_messages

# Usage example
filename = 'chat.txt'
filtered_messages = filter_whatsapp_chat(filename, sender_name="Vatsal", month="November", keyword="In")

# Display the filtered messages in a table format
if filtered_messages:
    print(tabulate(filtered_messages,
                   headers=['DateTime', 'Sender', 'Message', 'Category', 'Adjusted Date'],
                   tablefmt='grid'))
else:
    print("No messages found matching the criteria.")

+-------------------+----------+---------------------------------------------------------------------------------------------------------+------------+-----------------+
| DateTime          | Sender   | Message                                                                                                 | Category   | Adjusted Date   |
+===================+==========+=========================================================================================================+============+=================+
| 04/11/24 7:18 pm  | Vatsal   | filtered_messages = filter_whatsapp_chat(filename, sender_name="Vatsal", month="October", keyword="In") | Lunch      | 05/11/24        |
+-------------------+----------+---------------------------------------------------------------------------------------------------------+------------+-----------------+
| 13/11/24 4:55 pm  | Vatsal   | In                                                                                                      | Dinner     